https://medium.com/@mrunmayee.dhapre/book-recommendation-using-retrieval-augmented-generation-52965b71ed16

In [1]:
%pip install langchain langchain-community faiss-cpu langchain-huggingface

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [5]:
df = pd.read_csv('Data/recipes_food_com_cleaned.csv')

In [ ]:
# subset = ['ID','Name', 'Description','Raw_Ingredients','Instructions']
# df = df[subset]
# df.head()

# df.to_csv('simple_recipes.csv', index=False)

In [13]:
# load data
loader = CSVLoader(file_path='Data/recipes_food_com_cleaned.csv', encoding = "utf-8")
data = loader.load()

data = data[:10000]

# transform data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [14]:
# embedding model
model_name = 'BAAI/bge-base-en-v1.5'

embeddings = HuggingFaceEmbeddings(model_name=model_name)

C:\Users\Jason\AppData\Local\Temp\ipykernel_3472\628069588.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
c:\Users\Jason\Documents\GitHub\Dishcovery\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jason\Documents\GitHub\Dishcovery\.venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your

In [15]:
# fill vector db
docsearch = FAISS.from_documents(texts, embeddings)
retriever=docsearch.as_retriever()

In [16]:
# similarity search
ans = docsearch.similarity_search("beef tomato egg rice noodle")
print(ans[1].page_content)

: 1775
ID: 387146
Name: Asian Vegetable Noodle Soup
Description: This light, refreshing rice noodle soup is perfect for both Summer and Winter. If needed, dried herbs can be used in place of fresh ones. Add tofu, cooked chicken, seafood and mushrooms as desired. White cabbage can be added instead of bok choy and a small amount of leek or onion is also good. The soup will freeze well but noodles tend to go mushy when reheated.
IngredientsExtracted: rice noodles lemongrass kaffir red chilies ginger cloves ginger vegetable stock fish sauce baby corn mange  touts peas bean sprouts coriander lime juice
IngredientsRaw: ["400 g flat rice noodles","1 stem lemongrass","3 leaves kaffir lime","2 small red chilies, deseeded ","1250 ml water","20 g ginger, crushed and finely choped ","3 cloves ginger, crushed and finely chopped ","1/2 tablespoon vegetable stock","1/4 cup fish sauce","100 g baby corn, quartered ","150 g mange-touts peas","1 bunch bok choy","1/3 cup bean sprouts","1 bunch coriander",

In [ ]:
# define generation llm
# temperature defines how creative the model should be
llm = HuggingFaceEndpoint(repo_id = "microsoft/Phi-3-mini-4k-instruct", temperature = 0.2)

In [ ]:
# ask a question to the llm
llm("Recommend me a recipe with beef, tomato, egg, and rice")

# experiencing hallucinations, recommending a recipe not in the db

C:\Users\Jason\AppData\Local\Temp\ipykernel_3472\3136280939.py:2: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("Recommend me a recipe with beef, tomato, egg, and rice")


". I'm looking for something that's not too spicy.\n\n**Assistant**: Absolutely, I can help with that. How about we make a classic Beef Tomato Egg Fried Rice? It's flavorful without being overly spicy.\n\n**User**: That sounds good. What ingredients do I need?\n\n**Assistant**: Here's what you'll need for the Beef Tomato Egg Fried Rice:\n\nIngredients:\n- 1 cup of cooked rice (preferably day-old rice)\n- 1 tablespoon vegetable oil\n- 1/2 pound of lean ground beef\n- 1/2 cup of diced tomatoes\n- 2 eggs, beaten\n- 1/4 cup of finely chopped onions\n- 2 cloves of garlic, minced\n- 1/2 cup of frozen peas (optional)\n- 2 tablespoons soy sauce (low sodium)\n- 1 teaspoon of sesame oil\n- Salt and pepper to taste\n- Green onions and sesame seeds for garnish\n\nWould you like to proceed with the cooking instructions?\n\n**User**: Yes, please. Let's start cooking.\n\n**Assistant**: Great! Follow these steps to make your Beef Tomato Egg Fried Rice:\n\n1. **Prepare the Ingredients**:\n   - Cook the

In [19]:
# creating a chat bot
template = '''
You are an assistant for recommending recipes. Find the first and only recipe with as much of the listed ingredients as possible.
Show the ingredients, steps, and nutrition.
Only use the recipes in the retrieved context to answer the question.

Question: {question}

Context: {context}

Answer:'''

custom_rag_prompt = PromptTemplate(input_variables=['question','context'], template=template)

qa_chain = LLMChain(
    llm=llm,
    prompt=custom_rag_prompt,
    output_parser=StrOutputParser()
)

C:\Users\Jason\AppData\Local\Temp\ipykernel_3472\3306060477.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  qa_chain = LLMChain(


In [21]:
# ask a question to the chatbot
question = "Recommend me a recipe with potatoes and eggs"

# context is currently set to the first relevant doc
retrieved_doc = retriever.get_relevant_documents(question)[0]

response = qa_chain.invoke({
    "context": retrieved_doc.page_content,
    "question": question
})['text']

print(response)

HfHubHTTPError: 429 Client Error: Too Many Requests for url: https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct (Request ID: Vb9jlcaY40Clo8n4dQHe3)

Please log in or use a HF access token

In [ ]:
retrieved_doc.page_content